In [ ]:
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly_arrows as plar
init_notebook_mode(connected=True)

In [ ]:
c = 3e8 # Speed of light
w_conversion = 6.92e5 # Factor to make plot wavelength reasonable

<img title="Nomally incident EM wave" src="normal-incidence.png" width="550px">

The equations governing this interaction are

$$\mathbf{E_i} = E_{i,0} e^{i (\mathbf{k}_i \cdot \mathbf{r} - \omega_i t)}, \qquad \mathbf{E_t} = E_{t,0} e^{i (\mathbf{k}_t \cdot \mathbf{r} - \omega_t t)}, \qquad \mathbf{E_r} = E_{r,0} e^{i (\mathbf{k}_i \cdot \mathbf{r} - \omega_r t)}$$

N.B. the reflected wave propagates in the $-z$ direction

Taking the dielectric boundary to be at $z=0$, and applying continuity of $E_\parallel$ across the boundary,

$$E_i + E_r = E_t$$
$$\Rightarrow E_{i,0} e^{-i \omega_i t} + E_{r,0} e^{-i \omega_r t} + E_{t,0} e^{-i \omega_i t}$$

For the waves to be phase-matched at all times, the time-varying parts must always be equal, which can only happen if

$$E_{i,0} + E_{r,0} = E_{t,0}$$

Note that the wavenumber $k$ is not generally the same across the boundary.

As $B = \frac{k}{\omega} E$, the B field continuity is

$$B_{i,0} - B_{r,0} = B_{t,0}$$

But, $B = \frac{\eta}{c}E$

$$\Rightarrow \eta_1 E_{i,0} - \eta_1 E_{r,0} = \eta_2 E_{t,0}$$

So the ratio of transmitted to incident field is

$$t = \frac{E_{t,0}}{E_{i,0}} = \frac{\eta_1}{\eta_1 + \eta_2}$$

And the ratio of reflected to incident field is

$$r = \frac{E_{r,0}}{E_{i,0}} = \frac{\eta_1 - \eta_2}{\eta_1 + \eta_2}$$

Reflected ray satisfies $$\theta_i = \theta_r$$

Transmitted ray satisfies Snell's law $$\eta_1 \sin{\theta_i} = \eta_2 \sin{\theta_t}$$

<img src="fresnel-incidence.png" width="550px">

The full Fresnel equations are as follows

#### s-polarisation

$$t = \frac{E_t}{E_i} = \frac{2 \eta_1 \cos{\theta_i}}{\eta_1 \cos{\theta_i} + \eta_2 \cos{\theta_t}}$$

$$r = \frac{E_r}{E_i} = \frac{\eta_1 \cos{\theta_i} - \eta_2 \cos{\theta_t}}{\eta_1 \cos{\theta_i} + \eta_2 \cos{\theta_t}}$$

#### p-polarisation

$$t = \frac{E_t}{E_i} = \frac{2 \eta_1 \cos{\theta_i}}{\eta_1 \cos{\theta_t} + \eta_2 \cos{\theta_i}}$$

$$r = \frac{E_r}{E_i} = \frac{\eta_1 \cos{\theta_t} - \eta_2 \cos{\theta_i}}{\eta_1 \cos{\theta_t} + \eta_2 \cos{\theta_i}}$$

In [ ]:
class Wave:
    def __init__(self, theta, phi, out, E_0, polarisation, w=3.46e15, n1=1., width=2, color="rgb(0,0,0)"):
        """
        Args:
            theta (float) - [radians] {0 to π}
            phi (float) - [radians] {0 to 2π}
            out (bool) - True if outgoing, False if incoming (to the origin)
            
            E_0 (float) - Magnitude of the E field
            polarisation (str) - {'s'/'p'} whether E or B is parallel to the boundary
            w (float) - [rad s^-1] Angular frequency (default: green light)
            n1 (float) - The incident material's refractive index
            
            width (int) - line thickness
            color (str) - [hex/rgb] line color
        """
        self.theta = theta
        self.phi = phi
        if self.phi != 0:
            raise NotImplementedError("Use a phi angle of zero for now")
        self.out = out
        
        self.n1 = n1
        self.E_0 = E_0
        self.w = w / w_conversion
        self.true_w = w
        self.k = (n1 * self.w) / c
        self.true_k = (n1 * self.true_w) / c
        self.B_0 = self.n1 * E_0
        self.true_B_0 = (self.n1 / c) * E_0
        if polarisation == "s" or polarisation == "p":
            self.polarisation = polarisation
        else:
            raise Exception('Polarisation argument must be "s" or "p"')
            
        self.arrow = plar.Arrow(theta=self.theta, phi=self.phi, out=self.out,  width=width, color=color)
        self.sinusoids = self.create_sinusoids()
        
        self.width = width
        self.color = color
    
    
    def create_sinusoids(self):
        """Creates the sinusoidal wave components"""
        z_range = np.linspace(0, 1, 100)
        if self.polarisation == "s":
            E_sine = np.array([np.zeros_like(z_range), self.E_0 * -np.sin(self.k * z_range), z_range])
            B_sine = np.array([self.B_0 * -np.sin(self.k * z_range), np.zeros_like(z_range), z_range])
        else:
            E_sine = np.array([self.E_0 * np.sin(self.k * z_range), np.zeros_like(z_range), z_range])
            B_sine = np.array([np.zeros_like(z_range), self.B_0 * -np.sin(self.k * z_range), z_range])
        
        rot_E_sine = self.rotate_sinusoid(E_sine, self.theta, self.phi)
        rot_B_sine = self.rotate_sinusoid(B_sine, self.theta, self.phi)
        
        E_trace = go.Scatter3d(x=rot_E_sine[0], y=rot_E_sine[1], z=rot_E_sine[2],
                               mode = 'lines', marker = dict(color='#0099FF'))
        B_trace = go.Scatter3d(x=rot_B_sine[0], y=rot_B_sine[1], z=rot_B_sine[2],
                               mode = 'lines', marker = dict(color='#FF0099'))
        
        return [E_trace, B_trace]
        
    
    def transmit(self, n2):
        """
        Args:
            n2 (float) - The dielectric material's refactive index
        """
        self.n2 = n2
        
        theta_i = self.theta
        theta_t = self.snell(self.n1, self.n2, theta_i)
        plot_theta_t = np.pi + theta_t
        
        if self.polarisation == "s":
            E_t0 = self.E_0 * (2. * self.n1 * np.cos(theta_i)) / (self.n1 * np.cos(theta_i) + self.n2 * np.cos(theta_t))
        else:
            E_t0 = self.E_0 * (2. * self.n1 * np.cos(theta_i)) / (self.n1 * np.cos(theta_t) + self.n2 * np.cos(theta_i))
                
        return Wave(theta=plot_theta_t, phi=self.phi, out=True, E_0=E_t0, w=self.true_w,
                    polarisation=self.polarisation, n1=self.n2, color="#000000")
    
        
    def reflect(self, n2):
        """
        Args:
            n2 (float) - The dielectric material's refactive index
        """
        self.n2 = n2
        
        theta_i = self.theta
        theta_r = theta_i
        theta_t = self.snell(self.n1, self.n2, theta_i)
        plot_theta_r = -theta_r
        
        if self.polarisation == "s":
            E_r0 = self.E_0 * (self.n1 * np.cos(theta_i) - self.n2 * np.cos(theta_t)) / (self.n1 * np.cos(theta_i) + self.n2 * np.cos(theta_t))
        else:
            E_r0 = self.E_0 * (self.n1 * np.cos(theta_t) - self.n2 * np.cos(theta_i)) / (self.n1 * np.cos(theta_t) + self.n2 * np.cos(theta_i))
        
        return Wave(theta=plot_theta_r, phi=self.phi, out=True, E_0=E_r0, w=self.true_w,
                    polarisation=self.polarisation, n1=self.n1)
    
    
    @staticmethod
    def snell(n1, n2, theta_i):
        """
        Finds angle of transmission using Snell's law
        
        Args:
            n1 (float) - incident medium's refractive index
            n2 (float) - transmissive medium's refractive index
            theta_i (float) - angle of incidence
        """
        return np.arcsin((n1 / n2) * np.sin(theta_i))
    
    @staticmethod
    def rotate_sinusoid(sinusoid, theta, phi):
        rotation_matrix = np.array([[np.cos(theta) * np.cos(phi), -np.sin(phi),  np.cos(phi) * np.sin(theta)],
            [np.cos(theta) * np.sin(phi), np.cos(phi), np.sin(theta) * np.sin(phi)],
            [-np.sin(theta), 0, np.cos(theta)]])
        # Apply rotation matrix element-wise to array of sine position vectors (using Einstein summation convention):
        return np.einsum("ij,jk->ik", rotation_matrix, sinusoid)
        


In [ ]:
import pandas as pd

Incident = Wave(theta=np.deg2rad(60.), phi=0, out=False, E_0=0.2, polarisation="s", n1=1.)
Transmitted = Incident.transmit(n2=1.5)
Reflected = Incident.reflect(n2=1.5)

df = pd.DataFrame.from_items(
    [('Incident', [Incident.E_0, Incident.true_B_0, Incident.true_k, Incident.true_w, Incident.n1, Incident.polarisation]),
     ('Transmitted', [Transmitted.E_0, Transmitted.true_B_0, Transmitted.true_k, Transmitted.true_w, Transmitted.n1, Transmitted.polarisation]),
     ('Reflected', [Reflected.E_0, Reflected.true_B_0, Reflected.true_k, Reflected.true_w, Reflected.n1, Reflected.polarisation])],
    orient='index', columns=['E_0', 'B_0', 'k', 'w', 'n1', 'polarisation']
)
print(df)

In [ ]:
plot_data = Incident.arrow.data + Incident.sinusoids \
    + Transmitted.arrow.data + Transmitted.sinusoids \
    + Reflected.arrow.data + Reflected.sinusoids

layout = {
    'autosize': True,
    'width': 700, 'height': 700,
    'scene': {
        'aspectmode': 'cube',
        'xaxis': {'range': [-1, 1], 'autorange': False, 'zeroline': True},
        'yaxis': {'range': [-1, 1], 'autorange': False, 'zeroline': True},
        'zaxis': {'range': [-1, 1], 'autorange': False, 'zeroline': True},
        'camera': {
            'up': {'x': 0, 'y': 1, 'z': 0} # DOESN'T WORK -- WHY NOT!?
        }
    }
}

fig = go.Figure(data=plot_data, layout=layout)
iplot(fig)